# Naive Bayes (the easy way)
Vamos trapacear usando sklearn.naive_bayes para treinar um classificador de spam! A maior parte do código está apenas carregando nossos dados de treinamento em um DataFrame do pandas com o qual podemos brincar:

In [3]:
import os  # Biblioteca para interagir com o sistema de arquivos
import io  # Biblioteca para manipulação de entrada e saída
import numpy  # Biblioteca para computação numérica
import pandas as pd  # Biblioteca para manipulação de dados
from pandas import DataFrame  # Importa a classe DataFrame do pandas
from sklearn.feature_extraction.text import CountVectorizer  # Para converter texto em contagem de palavras
from sklearn.naive_bayes import MultinomialNB  # Para o modelo Naive Bayes

def readFiles(path):
    """
    Função que lê arquivos de texto em um diretório e retorna o caminho e o conteúdo das mensagens.
    """
    for root, dirnames, filenames in os.walk(path):  # Percorre o diretório especificado
        for filename in filenames:  # Para cada arquivo encontrado
            path = os.path.join(root, filename)  # Constrói o caminho completo do arquivo

            inBody = False  # Flag para indicar se estamos no corpo do email
            lines = []  # Lista para armazenar as linhas do email
            f = io.open(path, 'r', encoding='latin1')  # Abre o arquivo com codificação latin1
            for line in f:  # Lê o arquivo linha por linha
                if inBody:  # Se já passamos do cabeçalho
                    lines.append(line)  # Adiciona a linha ao corpo
                elif line == '\n':  # Se encontramos uma linha vazia
                    inBody = True  # Mudamos para o modo corpo
            f.close()  # Fecha o arquivo
            message = '\n'.join(lines)  # Junta as linhas do corpo em uma única string
            yield path, message  # Retorna o caminho e a mensagem

def dataFrameFromDirectory(path, classification):
    """
    Função que cria um DataFrame a partir de emails em um diretório especificado.
    """
    rows = []  # Lista para armazenar as linhas do DataFrame
    index = []  # Lista para armazenar os índices (caminhos dos arquivos)
    for filename, message in readFiles(path):  # Lê os arquivos do diretório
        rows.append({'message': message, 'class': classification})  # Adiciona a mensagem e a classificação
        index.append(filename)  # Adiciona o caminho do arquivo como índice

    return DataFrame(rows, index=index)  # Retorna um DataFrame construído

# Cria um DataFrame vazio com colunas 'message' e 'class'
data = DataFrame({'message': [], 'class': []})

# Concatena os DataFrames de spam e ham ao DataFrame principal
data = pd.concat([data, dataFrameFromDirectory("emails/spam", "spam")])
data = pd.concat([data, dataFrameFromDirectory("emails/ham", "ham")])

# Para versões antigas do Pandas (antes da versão 1.3), você poderia usar o seguinte:
# data = data.append(dataFrameFromDirectory('emails/spam', 'spam'))
# data = data.append(dataFrameFromDirectory('emails/ham', 'ham'))


In [4]:
data.head()

,message,class
emails/spam\00001.7848dde101aa985090474a91ec93fcf0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam
emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
emails/spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
emails/spam\00004.eac8de8d759b7e74154f142194282724,##############################################...,spam
emails/spam\00005.57696a39d7d84318ce497886896bf90d,I thought you might like these:\n\n1) Slim Dow...,spam


In [5]:
from sklearn.feature_extraction.text import CountVectorizer  # Para converter textos em contagens de palavras
from sklearn.naive_bayes import MultinomialNB  # Para o modelo Naive Bayes

# Cria um vetor de contagem que transforma textos em uma matriz de contagem de palavras
vectorizer = CountVectorizer()
# Aplica o vetor de contagem às mensagens dos emails e transforma em uma matriz esparsa
counts = vectorizer.fit_transform(data['message'].values)

# Inicializa o classificador Naive Bayes
classifier = MultinomialNB()
# Extrai os alvos (classes) do DataFrame
targets = data['class'].values
# Treina o classificador usando a matriz de contagem e os alvos
classifier.fit(counts, targets)


MultinomialNB()

## Activity
Nosso conjunto de dados é pequeno, então nosso classificador de spam não é muito bom. Tente executar alguns e-mails de teste diferentes e veja se obtém os resultados esperados.

Se você realmente deseja se desafiar, tente aplicar treinamento/teste a esse classificador de spam - veja como ele pode prever algum subconjunto de e-mails de spam e spam.

In [6]:
from sklearn.model_selection import train_test_split

# Divide os dados em conjuntos de treinamento e teste (80% para treino, 20% para teste)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Cria a matriz de contagem para os dados de treinamento
train_counts = vectorizer.fit_transform(train_data['message'].values)

# Extrai os alvos (classes) para o conjunto de treinamento
train_targets = train_data['class'].values

# Treina o classificador com os dados de treinamento
classifier.fit(train_counts, train_targets)

# Cria a matriz de contagem para os dados de teste
test_counts = vectorizer.transform(test_data['message'].values)

# Extrai os alvos (classes) para o conjunto de teste
test_targets = test_data['class'].values


In [7]:
from sklearn.metrics import accuracy_score, classification_report

# Faz previsões com o conjunto de teste
predictions = classifier.predict(test_counts)

# Avalia a precisão
accuracy = accuracy_score(test_targets, predictions)
print(f'Accuracy: {accuracy:.2f}')

# Mostra o relatório de classificação
print(classification_report(test_targets, predictions))


Accuracy: 0.95
              precision    recall  f1-score   support

         ham       0.94      1.00      0.97       482
        spam       0.98      0.76      0.86       118

    accuracy                           0.95       600
   macro avg       0.96      0.88      0.91       600
weighted avg       0.95      0.95      0.95       600



In [8]:
# Novos e-mails para testar
new_emails = [
    "Congratulations! You've won a lottery. Click here to claim your prize.",
    "Hi John, can we reschedule our meeting for next week?",
    "You have a new message from your bank regarding your account.",
    "This is a reminder for your upcoming appointment tomorrow."
]

# Transformar os novos e-mails em contagem de palavras
new_counts = vectorizer.transform(new_emails)

# Fazer previsões com os novos e-mails
new_predictions = classifier.predict(new_counts)

# Exibir as previsões
for email, prediction in zip(new_emails, new_predictions):
    print(f'Email: "{email}" - Prediction: {prediction}')


Email: "Congratulations! You've won a lottery. Click here to claim your prize." - Prediction: ham
Email: "Hi John, can we reschedule our meeting for next week?" - Prediction: ham
Email: "You have a new message from your bank regarding your account." - Prediction: spam
Email: "This is a reminder for your upcoming appointment tomorrow." - Prediction: ham
